# Advent of Code 2015

My solutions for [Advent of Code 2015](https://adventofcode.com/2015), based on Norvig [notebook](https://github.com/norvig/pytudes/blob/master/ipynb/Advent-2020.ipynb)

# Day 0: Imports and Utility Functions

Preparations prior to Day 1:
- Some imports.
- A way to read the day's data file and to print/check the output.
- Some utilities that are likely to be useful.

In [692]:
from itertools   import chain, combinations
from functools   import reduce
from hashlib     import md5
from typing      import Tuple, List, Set, Union, Callable
from contextlib  import contextmanager

import operator
import re
import numpy as np

In [693]:
def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    with open(f'AoC-2015/data/input-{day}') as f:
        sections = f.read().rstrip().split(sep)
        return list(map(parser, sections))

def do(day, *answers) -> List[int]:
    "E.g., do(3) returns [day3_1(in3), day3_2(in3)]. Verifies `answers` if given."
    g = globals()
    got = []
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g:
            got.append(g[fname](g[f'in{day}']))
            if len(answers) >= part:
                assert got[-1] == answers[part - 1], (
                    f'{fname}(in{day}) got {got[-1]}; expected {answers[part - 1]}')
        else:
            got.append(None)
    return got

In [694]:
Number = Union[float, int]
Atom = Union[Number, str]
Char = str # Type used to indicate a single character

cat = ''.join
flatten = chain.from_iterable

def quantify(iterable, pred=bool) -> int:
    "Count the number of items in iterable for which pred is true."
    return sum(1 for item in iterable if pred(item))

def first(iterable, default=None) -> object:
    "Return first item in iterable, or default."
    return next(iter(iterable), default)

def prod(numbers) -> Number:
    "The product of an iterable of numbers."
    return reduce(operator.mul, numbers, 1)

def dot(A, B) -> Number:
    "The dot product of two vectors of numbers."
    return sum(a * b for a, b in zip(A, B))

def ints(text: str) -> Tuple[int]:
    "Return a tuple of all the integers in text."
    return mapt(int, re.findall('-?[0-9]+', text))

def lines(text: str) -> List[str]:
    "Split the text into a list of lines."
    return text.strip().splitlines()

def mapt(fn, *args):
    "Do map(fn, *args) and make the result a tuple."
    return tuple(map(fn, *args))

def atoms(text: str, ignore=r'', sep=None) -> Tuple[Union[int, str]]:
    "Parse text into atoms separated by sep, with regex ignored."
    text = re.sub(ignore, '', text)
    return mapt(atom, text.split(sep))

def atom(text: str, types=(int, str)):
    "Parse text into one of the given types."
    for typ in types:
        try:
            return typ(text)
        except ValueError:
            pass

@contextmanager
def binding(**kwds):
    "Bind global variables within a context; revert to old values on exit."
    old_values = {k: globals()[k] for k in kwds}
    try:
        globals().update(kwds)
        yield # Stuff within the context gets run here.
    finally:
        globals().update(old_values)

Notes:
-  Since I'm not even attempting to compete to be among the first 100 people to find a solution, I'll take the time to write docstrings; to use reasonable variable names (not single-letter names); and to give type annotations for most functions (but not the day functions, which all return int, except day21_2).
-  Traditionally, a lot of AoC problems are solved by one of the following two forms:
  - quantify(inputs, P): How many of your input items have property P?
  - sum(map(F, inputs)): What is the sum of the result of applying F to each input item?
- Some days I will re-use code that was defined on a previous day.
- I will give a few tests using assert, but far fewer test cases than if I was programming seriously.

# Day 1: Not Quite Lisp
1. Find on which floor is Santa
2. Find when Santa enter the basement

In [695]:
in1: str = cat(data(1))

In [696]:
def day1_1(parentheses):
    """ Find on which floor is Santa """
    return parentheses.count("(") - parentheses.count(")")

In [697]:
def day1_2(parentheses):
    """ Find when Santa enter the basement """
    level = 0
    for i, parenthese in enumerate(parentheses):
        level += 1 if parenthese == "(" else -1
        if level == -1:
            return i + 1

In [698]:
do(1, 74, 1795)

[74, 1795]

# Day 2: I Was Told There Would Be No Math
1. How many square feet of wrapping paper ?
2. How many feet of ribbon ?

In [699]:
Dimensions = Tuple[int, int, int]

def parse_dimensions(line: str):
    return tuple(int(dimension) for dimension in line.split("x"))

in2: List[Dimensions] = data(2, parse_dimensions)

In [700]:
def day2_1(dimensions):
    """ How many square feet of wrapping paper ? """
    paper = 0
    for xyz in dimensions:
        sides = [s1 * s2 for s1, s2 in combinations(xyz, 2)]
        paper += sum(side * 2 for side in sides)
        paper += min(sides)
    return paper

In [701]:
def day2_2(dimensions):
    """ How many feet of ribbon ? """
    ribbon = 0
    for xyz in dimensions:
        xyz = sorted(xyz)
        ribbon += sum(xyz[:2]) * 2
        ribbon += prod(xyz)
    return ribbon

In [702]:
do(2, 1606483, 3842356)

[1606483, 3842356]

# Day 3: Perfectly Spherical Houses in a Vacuum
1. How many houses delivered by Santa ?
2. How many houses delivered by Santa and Robo-Santa ?

In [703]:
Directions = List[Char]
Coordinates = Tuple[int, int]

in3: Directions = cat(data(3))

In [704]:
def day3_1(directions):
    """ How many houses delivered by Santa ? """
    return len(deliver(directions))

def deliver(directions: Directions) -> Set[Coordinates]:
    cx, cy = 0, 0
    houses = {(cx, cy)}
    for direction in directions:
        if direction == ">":
            cx += 1
        elif direction == "<":
            cx -= 1
        elif direction == "^":
            cy += 1
        elif direction == "v":
            cy -= 1
        houses.add((cx, cy))
    return houses

In [705]:
def day3_2(directions):
    """ How many houses delivered by Santa and Robo-Santa ? """
    return len(deliver(directions[::2]) | deliver(directions[1::2]))

In [706]:
do(3, 2572, 2631)

[2572, 2631]

# Day 4: The Ideal Stocking Stuffer
1. Find md5 that starts with 5 zeroes.
2. Find md5 that starts with 6 zeroes.

In [707]:
in4: str = "ckczppom"

In [708]:
def day4_1(secret_key, n=5):
    i = 1
    while True:
        md5_value = md5(bytes(secret_key + str(i), encoding="utf8")).hexdigest()
        if md5_value.startswith("0"*n):
            return i
        i += 1

In [709]:
def day4_2(secret_key):
    return day4_1(secret_key, n=6)

In [710]:
do(4, 117946, 3938038)

[117946, 3938038]

# Day 5: Doesn't He Have Intern-Elves For This?
1. Count valid words.
2. Count valid words again.

In [711]:
in5 = data(5)

In [712]:
def day5_1(words):
    """ Count valid words """
    return quantify(words, validate1)

def validate1(word):
    vowels = len(re.findall("[aeiou]", word)) >= 3
    bad = re.search("ab|cd|pq|xy", word) is not None
    double = re.search("(?P<letter>.)(?P=letter)", word) is not None
    return vowels and not bad and double

In [713]:
def day5_2(words):
    """ Count valid words again """
    return quantify(words, validate2)

def validate2(word):
    twice = re.search("(?P<pair>..).*(?P=pair)", word) is not None
    repeat_between = re.search("(?P<letter>.).(?P=letter)", word) is not None
    return twice and repeat_between

In [714]:
do(5, 255, 55)

[255, 55]

# Day 6: Probably a Fire Hazard
1. How many lights are lit ?
2. What is the total brightness of all lights combined ?

In [715]:
Instruction = Tuple[str, int, int, int, int]

def instruction_parser(line):
    inst, x1, y1, x2, y2 = re.fullmatch(
        "(toggle|turn on|turn off) (\d+),(\d+) through (\d+),(\d+)",
        line,
    ).groups()
    return inst, int(x1), int(y1), int(x2), int(y2)

in6 = data(6, instruction_parser)

In [716]:
def day6_1(instructions):
    lights = np.full((1000, 1000), False)
    for inst, x1, y1, x2, y2 in instructions:
        if inst == "toggle":
            lights[x1:x2+1,y1:y2+1] = ~lights[x1:x2+1,y1:y2+1]
        if inst == "turn on":
            lights[x1:x2+1,y1:y2+1] = True
        if inst == "turn off":
            lights[x1:x2+1,y1:y2+1] = False
    return np.count_nonzero(lights)

In [717]:
def day6_2(instructions):
    lights = np.full((1000, 1000), 0)
    for inst, x1, y1, x2, y2 in instructions:
        if inst == "toggle":
            lights[x1:x2+1,y1:y2+1] += 2
        if inst == "turn on":
            lights[x1:x2+1,y1:y2+1] += 1
        if inst == "turn off":
            lights[x1:x2+1,y1:y2+1] -= 1
            lights[x1:x2+1,y1:y2+1] = lights[x1:x2+1,y1:y2+1].clip(min=0)
    return np.sum(lights)

In [718]:
do(6, 543903, 14687245)

[543903, 14687245]